In [1]:
import os
import pandas as pd
import numpy as np
import validation_data_input
import plotly.express as px
import toml
import psrc_theme

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
# model_dir = os.path.join(os.getcwd(), '..\\..\\..\\..\\')

In [2]:
hh = validation_data_input.get_data('household')
person = validation_data_input.get_data('person')

- person columns

In [3]:
person.columns

Index(['id', 'hhno', 'pno', 'pptyp', 'pagey', 'pgend', 'pwtyp', 'pwpcl',
       'pwtaz', 'pwautime', 'pwaudist', 'pstyp', 'pspcl', 'pstaz', 'psautime',
       'psaudist', 'puwmode', 'puwarrp', 'puwdepp', 'ptpass', 'ppaidprk',
       'pdiary', 'pproxy', 'psexpfac', 'source', 'worker_type'],
      dtype='object')

In [4]:
# add person type labels
ptype_cat = {1: "1: full time worker",
             2: "2: part time worker",
             3: "3: non-worker age 65+",
             4: "4: other non-working adult",
             5: "5: university student",
             6: "6: grade school student/child age 16+",
             7: "7: child age 5-15",
             8: "8: child age 0-4"}
person['pptyp_label'] = person['pptyp'].map(ptype_cat)

In [5]:
df_plot = person.groupby(['source','pptyp_label'])['psexpfac'].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['psexpfac'].\
        apply(lambda x: x / float(x.sum()))

df_plot_ct = person.groupby(['source','pptyp_label'])['psexpfac'].count().reset_index(). \
    rename(columns={'psexpfac':'sample count'})
df_plot = df_plot.merge(df_plot_ct, on=['source','pptyp_label'])

fig = px.bar(df_plot.sort_values(by=['source']), x="pptyp_label", y="percentage", color="source",
             hover_data=['sample count'],
             barmode="group",title="person type")
fig.update_layout(height=400, width=700, font=dict(size=11),
                  yaxis=dict(tickformat=".2%"))
fig.show()